# Portfolio Optimization and Backtesting Analysis

This notebook provides comprehensive portfolio optimization and backtesting capabilities using Modern Portfolio Theory. It includes:

- Interactive Efficient Frontier generation and portfolio selection
- Portfolio optimization using forecasting results
- Comprehensive backtesting workflow with performance analysis
- Strategy vs benchmark comparison and reporting

## Table of Contents
1. [Setup and Data Loading](#setup)
2. [Portfolio Optimization](#optimization)
3. [Portfolio Recommendations](#recommendations)
4. [Backtesting Analysis](#backtesting)
5. [Performance Comparison](#performance)
6. [Final Report](#report)

## 1. Setup and Data Loading {#setup}

In [ ]:
# Import libraries
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json

# Portfolio forecasting modules
from src.data.yfinance_client import YFinanceClient

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Configuration
ASSETS = ['TSLA', 'SPY', 'BND']
START_DATE = '2020-01-01'
END_DATE = '2024-12-31'
BACKTEST_START = '2024-08-01'
BACKTEST_END = '2025-07-31'
INITIAL_CAPITAL = 100000.0

print(f"📊 Assets: {ASSETS}")
print(f"📅 Data period: {START_DATE} to {END_DATE}")
print(f"🔄 Backtest period: {BACKTEST_START} to {BACKTEST_END}")
print(f"💰 Initial capital: ${INITIAL_CAPITAL:,.2f}")

In [ ]:
# Load market data
print("📥 Loading market data...")

client = YFinanceClient()
price_data = {}

for asset in ASSETS:
    try:
        data = client.fetch_data(asset, START_DATE, END_DATE)
        price_data[asset] = data['Close']
        print(f"✅ {asset}: {len(data)} observations loaded")
    except Exception as e:
        print(f"❌ Error loading {asset}: {str(e)}")

# Create combined DataFrame
df = pd.DataFrame(price_data).dropna()
print(f"\n📈 Combined dataset: {len(df)} observations")
display(df.tail())

In [ ]:
# Simple portfolio optimization
print("🚀 Portfolio Optimization")

# Calculate returns
returns = df.pct_change().dropna()

# Calculate expected returns and covariance
expected_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252

print("\n📊 Expected Annual Returns:")
for asset in ASSETS:
    print(f"  {asset}: {expected_returns[asset]:.2%}")

print("\n📈 Correlation Matrix:")
corr_matrix = returns.corr()
display(corr_matrix.round(3))

# Simple equal weight portfolio
equal_weights = np.array([1/len(ASSETS)] * len(ASSETS))
portfolio_return = np.sum(expected_returns * equal_weights)
portfolio_vol = np.sqrt(np.dot(equal_weights.T, np.dot(cov_matrix, equal_weights)))
sharpe_ratio = (portfolio_return - 0.02) / portfolio_vol

print(f"\n💼 Equal Weight Portfolio:")
print(f"  Expected Return: {portfolio_return:.2%}")
print(f"  Volatility: {portfolio_vol:.2%}")
print(f"  Sharpe Ratio: {sharpe_ratio:.3f}")

for i, asset in enumerate(ASSETS):
    print(f"  {asset}: {equal_weights[i]:.1%}")

In [ ]:
# Simple backtesting
print("📊 Simple Backtesting Analysis")

# Get backtest period data
backtest_data = df[BACKTEST_START:BACKTEST_END] if BACKTEST_START in df.index else df.tail(252)
backtest_returns = backtest_data.pct_change().dropna()

# Portfolio performance
portfolio_returns = (backtest_returns * equal_weights).sum(axis=1)
cumulative_returns = (1 + portfolio_returns).cumprod()

# Benchmark (SPY)
benchmark_returns = backtest_returns['SPY']
benchmark_cumulative = (1 + benchmark_returns).cumprod()

# Enhanced performance visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Portfolio Performance Analysis Dashboard', fontsize=16, fontweight='bold')

# Plot 1: Cumulative performance with metrics
ax1 = axes[0, 0]
portfolio_total_return = (cumulative_returns.iloc[-1] - 1) * 100
benchmark_total_return = (benchmark_cumulative.iloc[-1] - 1) * 100
outperformance = portfolio_total_return - benchmark_total_return

ax1.plot(cumulative_returns.index, cumulative_returns, 
         label=f'Portfolio ({portfolio_total_return:+.1f}%)', linewidth=3, color='#2E86AB')
ax1.plot(benchmark_cumulative.index, benchmark_cumulative, 
         label=f'SPY Benchmark ({benchmark_total_return:+.1f}%)', linewidth=3, color='#A23B72')

ax1.text(0.02, 0.98, f'Outperformance: {outperformance:+.1f}%', 
         transform=ax1.transAxes, fontsize=12, fontweight='bold',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
         verticalalignment='top')

ax1.set_title('Cumulative Performance Comparison', fontsize=14, fontweight='bold')
ax1.set_xlabel('Time Period')
ax1.set_ylabel('Cumulative Return')
ax1.legend(frameon=True, fancybox=True, shadow=True)
ax1.grid(True, alpha=0.3, linestyle='--')

# Plot 2: Rolling Sharpe ratios
ax2 = axes[0, 1]
rolling_sharpe_portfolio = portfolio_returns.rolling(60).mean() / portfolio_returns.rolling(60).std() * np.sqrt(252)
rolling_sharpe_benchmark = benchmark_returns.rolling(60).mean() / benchmark_returns.rolling(60).std() * np.sqrt(252)

ax2.plot(rolling_sharpe_portfolio.index, rolling_sharpe_portfolio, 
         label=f'Portfolio (Avg: {rolling_sharpe_portfolio.mean():.2f})', linewidth=2, color='#2E86AB')
ax2.plot(rolling_sharpe_benchmark.index, rolling_sharpe_benchmark, 
         label=f'Benchmark (Avg: {rolling_sharpe_benchmark.mean():.2f})', linewidth=2, color='#A23B72')

ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax2.axhline(y=1, color='green', linestyle='--', alpha=0.7, label='Good Performance')
ax2.set_title('Risk-Adjusted Performance (60-Day Rolling Sharpe)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Time Period')
ax2.set_ylabel('Sharpe Ratio')
ax2.legend(frameon=True, fancybox=True, shadow=True)
ax2.grid(True, alpha=0.3, linestyle='--')

# Plot 3: Drawdown analysis
ax3 = axes[1, 0]
portfolio_drawdown = (cumulative_returns / cumulative_returns.expanding().max() - 1) * 100
benchmark_drawdown = (benchmark_cumulative / benchmark_cumulative.expanding().max() - 1) * 100

ax3.fill_between(portfolio_drawdown.index, portfolio_drawdown, 0, 
                 alpha=0.4, color='#2E86AB', label=f'Portfolio (Max: {portfolio_drawdown.min():.1f}%)')
ax3.fill_between(benchmark_drawdown.index, benchmark_drawdown, 0, 
                 alpha=0.4, color='#A23B72', label=f'Benchmark (Max: {benchmark_drawdown.min():.1f}%)')

ax3.axhline(y=-5, color='orange', linestyle='--', alpha=0.7, label='Moderate Risk (-5%)')
ax3.axhline(y=-10, color='red', linestyle='--', alpha=0.7, label='High Risk (-10%)')
ax3.set_title('Drawdown Analysis', fontsize=14, fontweight='bold')
ax3.set_xlabel('Time Period')
ax3.set_ylabel('Drawdown (%)')
ax3.legend(frameon=True, fancybox=True, shadow=True)
ax3.grid(True, alpha=0.3, linestyle='--')

# Plot 4: Return distributions
ax4 = axes[1, 1]
portfolio_monthly = (portfolio_returns + 1).resample('M').prod() - 1
benchmark_monthly = (benchmark_returns + 1).resample('M').prod() - 1

ax4.hist(portfolio_monthly * 100, bins=20, alpha=0.7, 
         label=f'Portfolio (μ={portfolio_monthly.mean()*100:.1f}%, σ={portfolio_monthly.std()*100:.1f}%)', 
         color='#2E86AB', density=True, edgecolor='white')
ax4.hist(benchmark_monthly * 100, bins=20, alpha=0.7, 
         label=f'Benchmark (μ={benchmark_monthly.mean()*100:.1f}%, σ={benchmark_monthly.std()*100:.1f}%)', 
         color='#A23B72', density=True, edgecolor='white')

ax4.axvline(x=0, color='black', linestyle='-', alpha=0.5)
ax4.set_title('Monthly Return Distribution', fontsize=14, fontweight='bold')
ax4.set_xlabel('Monthly Return (%)')
ax4.set_ylabel('Probability Density')
ax4.legend(frameon=True, fancybox=True, shadow=True)
ax4.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

# Performance metrics
total_return_portfolio = cumulative_returns.iloc[-1] - 1
total_return_benchmark = benchmark_cumulative.iloc[-1] - 1
portfolio_vol_backtest = portfolio_returns.std() * np.sqrt(252)
benchmark_vol_backtest = benchmark_returns.std() * np.sqrt(252)

print(f"\n📈 Backtest Results:")
print(f"Portfolio Total Return: {total_return_portfolio:.2%}")
print(f"Benchmark Total Return: {total_return_benchmark:.2%}")
print(f"Outperformance: {total_return_portfolio - total_return_benchmark:.2%}")
print(f"Portfolio Volatility: {portfolio_vol_backtest:.2%}")
print(f"Benchmark Volatility: {benchmark_vol_backtest:.2%}")

print("\n✅ Portfolio optimization and backtesting completed!")